<a href="https://colab.research.google.com/github/AshHydroCoder/GRACE-Downscaler/blob/Branch1/Downloading_Gap_filled_LST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connecting GEE with Google Services

- **Authenticate to Earth Engine**

In [1]:
!pip install earthengine-api #earth-engine Python API

In [2]:
!earthengine authenticate

Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
W0106 00:47:49.428903 132148884025344 _default.py:683] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=N0LL6MAMx2VDDayj-pQ8i-gBmP-cpuhHduLEf1Q-2iw&tc=u4t1u0HwUJqcrsGosvSwcLmxKa9nK89cD66KDEFvJm4&cc=j6dVuZ3KCxxmFQpPBD6xJ_nPzJi9l2-P2qUZlHuVVDI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AanRRrv1PDsCH1GiKXRWnwW6Jpsf

- **Authenticate to Google Drive**


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- **Authenticate to Google Cloud**

In [4]:
from google.colab import auth
auth.authenticate_user()

## Testing the software setup

In [5]:
# Earth Engine Python API
import ee
ee.Initialize()

In [6]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.19.2


In [7]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1. Exporting images

You can export images from Earth Engine in [GeoTIFF](https://trac.osgeo.org/geotiff/) or [TFRecord](https://www.tensorflow.org/api_docs/python/tf/io#TFRecords_Format_Details) format. See [Configuration Parameters](https://developers.google.com/earth-engine/exporting#configuration-parameters) for more output options.


In [8]:
import ee
ee.Initialize()

# Define the ImageCollection
gf_day_1km = ee.ImageCollection("projects/sat-io/open-datasets/gap-filled-lst/gf_day_1km")

# Define the region of interest (ROI) and date range
shape = ee.FeatureCollection("projects/ee-079mswre006ashish/assets/Nepal_Boundary")
start_date = '2012-11-01'
end_date = '2016-12-31'

# Filter the collection by date and bounds
filtered_collection = gf_day_1km.filterDate(start_date, end_date).filterBounds(shape)
# Convert the ImageCollection to a list
image_list = filtered_collection.toList(filtered_collection.size())

# Define a function to export each image
def export_image(image, index):
    image = ee.Image(image)  # Convert to an Image object
    # Get image date and format it to be valid for description and fileNamePrefix
    image_date = image.date().format('yyyyMMdd').getInfo()

    # Create a valid description and fileNamePrefix - removing hyphens
    file_name = f"LST_{image_date}"
    description = f"LST_{image_date}"

    region = shape.geometry().bounds().coordinates().get(0).getInfo()
    task = ee.batch.Export.image.toDrive(
        image=image.clip(shape),
        description=description,  # Using the modified description
        folder="GEE_Exports",  # Specify your Google Drive folder
        fileNamePrefix=file_name,  # Using the modified file name prefix
        scale=1000,  # Adjust scale as needed
        region=region,
        maxPixels=1e13  # Adjust max pixels if necessary
    )
    task.start()
    print(f"Exporting {file_name}...")

# Loop through the images and export them
for i in range(filtered_collection.size().getInfo()):  # Ensure iteration over the correct size
    export_image(image_list.get(i), i)

Exporting LST_20121101...
Exporting LST_20121102...
Exporting LST_20121103...
Exporting LST_20121104...
Exporting LST_20121105...
Exporting LST_20121106...
Exporting LST_20121107...
Exporting LST_20121108...
Exporting LST_20121109...
Exporting LST_20121110...
Exporting LST_20121111...
Exporting LST_20121112...
Exporting LST_20121113...
Exporting LST_20121114...
Exporting LST_20121115...
Exporting LST_20121116...
Exporting LST_20121117...
Exporting LST_20121118...
Exporting LST_20121119...
Exporting LST_20121120...
Exporting LST_20121121...
Exporting LST_20121122...
Exporting LST_20121123...
Exporting LST_20121124...
Exporting LST_20121125...
Exporting LST_20121126...
Exporting LST_20121127...
Exporting LST_20121128...
Exporting LST_20121129...
Exporting LST_20121130...
Exporting LST_20121201...
Exporting LST_20121202...
Exporting LST_20121203...
Exporting LST_20121204...
Exporting LST_20121205...
Exporting LST_20121206...
Exporting LST_20121207...
Exporting LST_20121208...
Exporting LS

In [9]:
!pip install rasterio
import os
import rasterio
import numpy as np
from rasterio.merge import merge

# Define input folder containing daily LST `.tif` files
input_folder = "/content/drive/MyDrive/GEE_Exports"
output_folder = "/content/drive/MyDrive/Thesis/RAW/LST/"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to parse the month from the filename
def extract_month(filename):
    date_part = filename.split("_")[1].split(".")[0]  # Assumes format LST_YYYYMMDD.tif
    return date_part[:6]  # Extract YYYYMM

# Group files by month
files_by_month = {}
for file in os.listdir(input_folder):
    if file.endswith(".tif"):
        month = extract_month(file)
        files_by_month.setdefault(month, []).append(os.path.join(input_folder, file))

# Process each month
for month, files in files_by_month.items():
    print(f"Processing month: {month} with {len(files)} files")

    # Read all rasters for the month
    rasters = []
    metadata = None
    for file in files:
        with rasterio.open(file) as src:
            rasters.append(src.read(1))  # Read the first (and only) band
            if metadata is None:
                metadata = src.meta

    # Convert list to 3D numpy array (time, rows, columns)
    raster_stack = np.stack(rasters, axis=0)

    # Calculate the monthly average
    monthly_average = np.mean(raster_stack, axis=0)

    # Update metadata for writing the average raster
    metadata.update({
        "count": 1,  # Single band output
        "dtype": "float32"  # Ensure output type is float32 for averages
    })

    # Write the monthly average raster to output folder
    output_file = os.path.join(output_folder, f"LST_{month}_avg.tif")
    with rasterio.open(output_file, "w", **metadata) as dest:
        dest.write(monthly_average.astype("float32"), 1)  # Write as the first band

    print(f"Saved monthly average raster: {output_file}")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.6 MB/s eta 0:00:00
Processing month: 201101 with 32 files


ValueError: all input arrays must have the same shape

In [ ]:
import os
folder_path = '/content/drive/MyDrive/GEE_Exports'
num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Number of files in '{folder_path}': {num_files}")

Number of files in '/content/drive/MyDrive/GEE_Exports': 2434
